In [1]:
pip install azure-storage-blob dotenv


   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ------------------ --------------------- 1.6/3.4 MB 7.6 MB/s eta 0:00:01
   ------------------------------------ --- 3.1/3.4 MB 7.4 MB/s eta 0:00:01
   ---------------------------------------- 3.4/3.4 MB 6.9 MB/s eta 0:00:00

   ---------------------------------------- 0/7 [pycparser]
   ---------------------------------------- 0/7 [pycparser]
   ----------- ---------------------------- 2/7 [dotenv]
   ----------------- ---------------------- 3/7 [cffi]
   ----------------- ---------------------- 3/7 [cffi]
   ---------------------- ----------------- 4/7 [azure-core]
   ---------------------- ----------------- 4/7 [azure-core]
   ---------------------- ----------------- 4/7 [azure-core]
   ---------------------- ----------------- 4/7 [azure-core]
   ---------------------- ----------------- 4/7 [azure-core]
   ---------------------- ----------------- 4/7 [azure-core]
   ---------------------------- ----------- 5/7 [

In [1]:
# importing necessary libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os
import datetime

In [2]:
#Data extraction from Azure Blob Storage

try:
    data = pd.read_csv(r'C:\Users\USER\Desktop\Zipco Casestudy with Airflow\zipco_transaction.csv')
    print("Data extracted successfully")
except Exception as e:
    print(f"Error extracting data: {e}")

Data extracted successfully


In [3]:
data.head()  # Display the first few rows of the dataframe

Date      ProductName  Quantity  UnitPrice StoreLocation  \
0  2023-01-01 00:00:00     Vanilla Cake         2  12.532304         South   
1  2023-01-01 01:00:00  Red Velvet Cake         1   7.083070         South   
2  2023-01-01 02:00:00   Chocolate Cake         5   6.736064         North   
3  2023-01-01 03:00:00      Carrot Cake         2   7.314823         North   
4  2023-01-01 04:00:00  Pizza Pepperoni         1   7.577727          East   

   PaymentType  PromotionApplied Weather  Temperature StaffPerformanceRating  \
0         Cash              True   Rainy    20.654914                   Poor   
1         Cash             False   Rainy    23.549497                Average   
2         Cash              True   Rainy          NaN              Excellent   
3         Cash             False  Cloudy    20.137483                   Poor   
4  Credit Card              True  Cloudy    23.020987                   Good   

   ... DeliveryTime_min     OrderType      CustomerName  \
0  ...               30      In Store     William Adams   
1  ...               33      In Store   Anthony Wiggins   
2  ...               43   Phone Order       Ashley Duke   
3  ...               32  Online Order    Brandon Taylor   
4  ...               58      In Store  Brittany Watkins   

                                     CustomerAddress Customer_PhoneNumber  \
0           9851 David Green\r\nTonyaburgh, VA 02853    (916)427-7276x861   
1      24682 Holly Stravenue\r\nMooreville, NH 13901         769.318.4373   
2  10184 Washington Trace Apt. 679\r\nEast Brandi...         702.520.3286   
3           87194 Jeff Rue\r\nMitchellbury, CA 50463         622-527-9530   
4             850 Julia Groves\r\nHartview, WI 95954         759-517-8359   

                   CustomerEmail        Staff_Name              Staff_Email  \
0             lisa00@example.net      John Bridges    pdavidson@example.com   
1  michellefernandez@example.com     Sarah Bentley     ajohnson@example.net   
2      cooperwilliam@example.com  Connie Cervantes    michele29@example.net   
3             fsilva@example.net   Jessica Stewart      xwilson@example.org   
4       petersjoseph@example.net  Cheryl Carpenter  christine96@example.org   

  DayOfWeek TotalSales  
0    Sunday  25.064608  
1    Sunday   7.083070  
2    Sunday  33.680321  
3    Sunday  14.629647  
4    Sunday   7.577727  

[5 rows x 21 columns]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [5]:
# Data Cleaning na Transformation
# remove duplicates entries

data.drop_duplicates(inplace=True)  

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             900 non-null    float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        900 non-null    object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [7]:
# replace missing values with mean for numerical columns
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    data.fillna({col:data[col].mean()}, inplace=True)

In [8]:
# replacing missing object values with 'Unknown'
object_cols = data.select_dtypes(include=['object']).columns
for col in object_cols:
    data.fillna({col:'Unknown'}, inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        1000 non-null   object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [10]:
# covert date columns to datetime format
Data = pd.to_datetime(data['Date'], errors='coerce')

 

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        1000 non-null   object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [12]:
data.columns


Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [15]:
#creating product table
products = data[['ProductName']].drop_duplicates().reset_index(drop=True)
products.index.name = 'ProductID'
products.reset_index(inplace=True)


In [16]:
products.head(20)  # Display the first few rows of the products dataframe

ProductID               ProductName
0          0              Vanilla Cake
1          1           Red Velvet Cake
2          2            Chocolate Cake
3          3               Carrot Cake
4          4           Pizza Pepperoni
5          5          Pizza Margherita
6          6  Pizza Beef Double decker
7          7                Cheesecake
8          8                Fruit Cake

In [18]:
#Creating Customer table
customers = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail']].drop_duplicates().reset_index(drop=True)
customers.index.name = 'CustomerID'
customers.reset_index(inplace=True)

In [19]:
customers.head()  # Display the first few rows of the customers dataframe 

CustomerID      CustomerName  \
0           0     William Adams   
1           1   Anthony Wiggins   
2           2       Ashley Duke   
3           3    Brandon Taylor   
4           4  Brittany Watkins   

                                     CustomerAddress Customer_PhoneNumber  \
0           9851 David Green\r\nTonyaburgh, VA 02853    (916)427-7276x861   
1      24682 Holly Stravenue\r\nMooreville, NH 13901         769.318.4373   
2  10184 Washington Trace Apt. 679\r\nEast Brandi...         702.520.3286   
3           87194 Jeff Rue\r\nMitchellbury, CA 50463         622-527-9530   
4             850 Julia Groves\r\nHartview, WI 95954         759-517-8359   

                   CustomerEmail  
0             lisa00@example.net  
1  michellefernandez@example.com  
2      cooperwilliam@example.com  
3             fsilva@example.net  
4       petersjoseph@example.net

In [21]:
#creating staff table
staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'StaffID'
staff.reset_index(inplace=True)

In [22]:
staff.head()  # Display the first few rows of the staff dataframe

StaffID        Staff_Name              Staff_Email
0        0      John Bridges    pdavidson@example.com
1        1     Sarah Bentley     ajohnson@example.net
2        2  Connie Cervantes    michele29@example.net
3        3   Jessica Stewart      xwilson@example.org
4        4  Cheryl Carpenter  christine96@example.org

In [70]:
# creating Trancsaction table
transactions = data.merge(products, on=['ProductName'], how='left')\
                   .merge(customers, on=['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail'], how='left')\
                   .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left') 

transactions.index.name = 'TransactionID'
transactions.reset_index(inplace=True)
transactions = transactions[['TransactionID','Date',  'ProductID', 'Quantity', 'UnitPrice', 'StoreLocation', 'PaymentType', 'PromotionApplied', 'Weather', 'Temperature', 'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min', 'OrderType', 'CustomerID', 'StaffID', 'DayOfWeek',
                             'TotalSales']] 
                             

                 
    
                


In [71]:
transactions.head()  # Display the first few rows of the transactions dataframe

TransactionID                 Date  ProductID  Quantity  UnitPrice  \
0              0  2023-01-01 00:00:00          0         2  12.532304   
1              1  2023-01-01 01:00:00          1         1   7.083070   
2              2  2023-01-01 02:00:00          2         5   6.736064   
3              3  2023-01-01 03:00:00          3         2   7.314823   
4              4  2023-01-01 04:00:00          4         1   7.577727   

  StoreLocation  PaymentType  PromotionApplied Weather  Temperature  \
0         South         Cash              True   Rainy    20.654914   
1         South         Cash             False   Rainy    23.549497   
2         North         Cash              True   Rainy    27.154342   
3         North         Cash             False  Cloudy    20.137483   
4          East  Credit Card              True  Cloudy    23.020987   

  StaffPerformanceRating CustomerFeedback  DeliveryTime_min     OrderType  \
0                   Poor          Neutral                30      In Store   
1                Average         Positive                33      In Store   
2              Excellent          Unknown                43   Phone Order   
3                   Poor         Positive                32  Online Order   
4                   Good          Neutral                58      In Store   

   CustomerID  StaffID DayOfWeek  TotalSales  
0           0        0    Sunday   25.064608  
1           1        1    Sunday    7.083070  
2           2        2    Sunday   33.680321  
3           3        3    Sunday   14.629647  
4           4        4    Sunday    7.577727

In [72]:
#saving the dataframes to CSV files
data.to_csv('cleaned_data.csv', index=False)
products.to_csv('products.csv', index=False)
customers.to_csv('customers.csv', index=False)
staff.to_csv('staff.csv', index=False)
transactions.to_csv('transactions.csv', index=False)